In [1]:
# Python program to HMM with 2 parts.
# Part1: figure out where the stationary "hidden" car is!
# The car and your agent (car) live in a nxn periodic grid world.
# assume a shape of car is square, length is 1

import numpy as np
import pandas as pd
import sys
from scipy.stats import norm
import math
import matplotlib.pyplot as plt
import matplotlib.colors as colors

In [2]:
# create discrete colormap to illustrate your belief
# dark red - most likely the position of the car!
# red - more likely the position of the car
# Gray level - belief level, darker = higher belief
def seeTrackedCar(gridIm):
    cmap = colors.ListedColormap(
        ['whitesmoke', 'gainsboro', 'silver', 'lightgray',
         'dimgray', 'gray', 'darkgray',  'red', 'darkred', 'orange'])
    bounds = [0, .1, .2, .3, .4, .5, .6, .7, .8, .9, 1.0]
    norm = colors.BoundaryNorm(bounds, cmap.N)
    fig, ax = plt.subplots()
    ax.imshow(gridIm, cmap=cmap, norm=norm)
    # draw gridlines
    ax.grid(which='major', axis='both', linestyle='-', color='k', linewidth=1)
    #
    #ax.invert_yaxis()
    ax.set_xticks(np.arange(0, gridIm.shape[1], 1))
    ax.set_yticks(np.arange(0, gridIm.shape[0], 1))
    plt.show()

In [3]:
# print the values stored on grid just in case you are interested in the numbers
# i = x = row,  j = y = column
def printGrid(grid):
    for i in range(grid.shape[0]):
        for j in range(grid.shape[1]-1):
            print(f"{grid[i][j]:.3f}, ", end="")
        print(f"{grid[i][grid.shape[1] - 1]:.3f}")


In [4]:
# Function: Get Belief
# ---------------------
# Updates beliefs based on recorded distance, transition prob, and your car's (agent's) pos.
# @input: gridSize, recording/observation, transition prob, car length = 1
# @return: your belief of the prob the "hidden" car is at each tile at each time step.
# Note: Your belief probabilities should sum to 1. (belief probabilities = posterior prob)
def getBeliefwMovingObj(N, observation, transitionP, carLength):
    std = carLength * (1/3)

    reportingTime = observation.shape[0]
    
    # tracking the car one frame/time step. carTrackingFrames is a collection of probMaps.
    carTrackingFrames = np.zeros((reportingTime+1, N, N))

    # your code
    prior = 1 / (N * N)
    carTrackingFrames[0] = np.ones((N, N)) * prior

    transitionTable = {}
    for _, row in transitionP.iterrows():
        x, y = int(row['X']), int(row['Y'])
        transitionTable[(x, y)] = {
            'N': row['N'],
            'E': row['E'],
            'S': row['S'],
            'W': row['W']}

    for t in range(1, observation.shape[0] + 1):
        prev_map = carTrackingFrames[t - 1]
        p_map = np.zeros((N, N))

        for x in range(N):
            for y in range(N):
                prob = prev_map[x, y]
                if prob == 0:
                    continue
                trans = transitionTable[(x, y)]
                p_map[(x - 1) % N, y] += prob * trans['N']
                p_map[x, (y + 1) % N] += prob * trans['E']
                p_map[(x + 1) % N, y] += prob * trans['S']
                p_map[x, (y - 1) % N] += prob * trans['W']

        agentX = int(observation.iloc[t - 1, 0])
        agentY = int(observation.iloc[t - 1, 1])
        eDist  = observation.iloc[t - 1, 2]

        for x in range(N):
            for y in range(N):
                dx = abs(agentX - x)
                dy = abs(agentY - y)
                dx = min(dx, (N - dx))
                dy = min(dy, (N - dy))

                distance = math.sqrt(dx**2 + dy**2)

                likehood = norm.pdf(eDist, distance, std)
                p_map[x, y] *= likehood

        total = np.sum(p_map)
        if total > 0:
            p_map /= total

        carTrackingFrames[t] = p_map

    return carTrackingFrames[1:]


In [5]:
# No need to change this function.
def hmm_part2(gridSize, reportingTime, microphoneReadingFileName, transitionProbFileName):
    
    carLength = 1
    transitionP = pd.read_csv(transitionProbFileName)
    
    readings_df = pd.read_csv(microphoneReadingFileName, nrows=reportingTime)
    
    probMapWithTime = getBeliefwMovingObj(gridSize, readings_df, transitionP, carLength)

    mostProbableCarPosWithTime = np.zeros([reportingTime, 2])
    secondProbableCarPosWithTime = np.zeros([reportingTime, 2])
    thirdProbableCarPosWithTime = np.zeros([reportingTime, 2])
    
    for t in range(reportingTime):
        mostProbableCarPosWithTime[t] = np.unravel_index(np.argmax(probMapWithTime[t], axis=None), probMapWithTime[t].shape)
        secondProbableCarPosWithTime[t] = np.unravel_index(np.argsort(probMapWithTime[t], axis=None)[-2], probMapWithTime[t].shape) 
        thirdProbableCarPosWithTime[t] = np.unravel_index(np.argsort(probMapWithTime[t], axis=None)[-3], probMapWithTime[t].shape) 

    df1 = pd.DataFrame(mostProbableCarPosWithTime, columns=['carX1', 'carY1'], dtype = np.int32)
    df2 = pd.DataFrame(secondProbableCarPosWithTime, columns=['carX2', 'carY2'], dtype = np.int32)
    df3 = pd.DataFrame(thirdProbableCarPosWithTime, columns=['carX3', 'carY3'], dtype = np.int32)

    df = pd.concat([df1, df2, df3], axis=1)
    
    fileName = "mostProbableLocationWithGrid" + str(gridSize) + "_tillTime" + str(reportingTime) + ".csv"
    print(fileName)
    df.to_csv(fileName, index=False)

    return

gridSize = 10
reportingTime = 100
microphoneReadingFileName = "movingCarReading10.csv"
transitionProbFileName = "transitionProb10.csv"
hmm_part2(gridSize, reportingTime, microphoneReadingFileName, transitionProbFileName)

mostProbableLocationWithGrid10_tillTime100.csv
